# Attribution and allocation

In [1]:
import pandas as pd
import numpy as np
# read the channel spend dataset
c_spend = pd.read_csv('channel_spend_graduate.csv')
c_spend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   channel    40 non-null     object
 1   date       40 non-null     int64 
 2   spend_AED  40 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ KB


In [2]:
# change the format of variable
c_spend['date'] = c_spend.date.map(str)
c_spend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   channel    40 non-null     object
 1   date       40 non-null     object
 2   spend_AED  40 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.1+ KB


In [3]:
# rewrite the format of datatime values
for i in range(c_spend.shape[0]):
    c_spend.iloc[i, 1] = '-'.join([c_spend.iloc[i, 1][:4],c_spend.iloc[i, 1][4:6]])

In [4]:
c_spend.head(2)

,channel,date,spend_AED
0,bing,2019-07,400
1,display,2019-07,12


In [5]:
c_spend['date'] = pd.to_datetime(c_spend['date'])

In [6]:
c_spend.info()
c_spend.to_csv('channel_spend.csv')   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   channel    40 non-null     object        
 1   date       40 non-null     datetime64[ns]
 2   spend_AED  40 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.1+ KB


In [7]:
# calculate total channel spend
total_c_s = c_spend.spend_AED.sum()
total_c_s

355896

In [8]:
# total spend of each channel
c_spend.groupby('channel')['spend_AED'].agg('sum')

channel
bing         10800
display        366
facebook    113500
search      222500
youtube       8730
Name: spend_AED, dtype: int64

# Compute channel spend

In [9]:
not_in_trial = pd.read_csv('not_in_trial.csv')

In [10]:
not_in_trial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218778 entries, 0 to 218777
Data columns (total 35 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    218778 non-null  int64  
 1   subid                         218778 non-null  int64  
 2   package_type                  183627 non-null  object 
 3   num_weekly_services_utilized  112599 non-null  float64
 4   preferred_genre               182870 non-null  object 
 5   intended_use                  215664 non-null  object 
 6   weekly_consumption_hour       182561 non-null  float64
 7   num_ideal_streaming_services  111141 non-null  float64
 8   age                           184880 non-null  float64
 9   male_TF                       218574 non-null  object 
 10  country                       218778 non-null  object 
 11  attribution_technical         218778 non-null  object 
 12  attribution_survey            216574 non-nul

In [11]:
sub_not_in_trial = not_in_trial.loc[:,['subid','account_creation_date','attribution_technical','attribution_survey','user_label','payment_period','discount_price']]

In [12]:
def convert_to_datetime(df, columns):
    for i in columns:
        df[i] = pd.to_datetime(df[i])
    return df
date_columns = ['account_creation_date']

In [13]:
sub_not_in_trial =convert_to_datetime(sub_not_in_trial, date_columns)

In [14]:
# filter channel spend months
channel_month = [1,2,7,8,9,10,11,12]
channel = sub_not_in_trial[sub_not_in_trial['account_creation_date'].dt.month.isin(channel_month)]

In [15]:
# filter channel_type based on the value of attribution_technical
channel_type = ['bing', 'search','youtube','display','facebook']
channel_s = channel[channel['attribution_technical'].isin(channel_type)]
channel_s.groupby('attribution_technical').size()

attribution_technical
bing          967
display      1259
facebook    72044
search      22105
youtube       734
dtype: int64

In [16]:
channel_s.user_label.unique()

array(['not_churn', 'never_sub', 'churn'], dtype=object)

In [17]:
cac_channel = channel_s[channel_s['user_label'] != 'never_sub']

In [18]:
# total converted users gained from each channel
cac_channel.groupby('attribution_technical').size()

attribution_technical
bing          506
display       721
facebook    40517
search      15017
youtube       583
dtype: int64

In [19]:
cac_channel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57344 entries, 0 to 218776
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   subid                  57344 non-null  int64         
 1   account_creation_date  57344 non-null  datetime64[ns]
 2   attribution_technical  57344 non-null  object        
 3   attribution_survey     57121 non-null  object        
 4   user_label             57344 non-null  object        
 5   payment_period         57344 non-null  int64         
 6   discount_price         57344 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 3.5+ MB


In [20]:
cac_channel.account_creation_date.describe()

<ipython-input-20-c509e7b6ed53>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  cac_channel.account_creation_date.describe()


count                   57344
unique                  57217
top       2019-09-22 13:32:28
freq                        2
first     2019-07-01 00:04:29
last      2020-02-29 23:56:41
Name: account_creation_date, dtype: object

In [21]:
cac_channel['months_per_bill_period'] = 4

<ipython-input-21-f5591eef9219>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cac_channel['months_per_bill_period'] = 4


In [22]:
cac_channel['month_year'] = cac_channel.account_creation_date.dt.to_period('M')

<ipython-input-22-598b136df804>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cac_channel['month_year'] = cac_channel.account_creation_date.dt.to_period('M')


# Calculate the marginal cac

In [23]:

month_user_channel = cac_channel.groupby(['month_year','attribution_technical']) \
                             .size() \
                             .reset_index(name='num_paying_user') \
                             #.sort_values(['count'], ascending=False) \
                             

In [24]:
month_user_channel.num_paying_user.sum()

57344

In [25]:
month_user_channel.rename(columns = {'attribution_technical':'channel'},inplace = True)
month_user_channel.head()

,month_year,channel,num_paying_user
0,2019-07,bing,25
1,2019-07,display,7
2,2019-07,facebook,2058
3,2019-07,search,457
4,2019-07,youtube,1


In [26]:
c_spend['month_year'] = c_spend.date.dt.to_period('M')

In [27]:
c_spend.drop(columns = 'date', inplace = True)
c_spend.head()

,channel,spend_AED,month_year
0,bing,400,2019-07
1,display,12,2019-07
2,facebook,9000,2019-07
3,search,13000,2019-07
4,youtube,90,2019-07


In [28]:
# merge two dataframes
new_cac = pd.merge(c_spend, month_user_channel, how='left', left_on=['channel','month_year'], right_on = ['channel','month_year'])

In [29]:
new_cac['month_cac'] = new_cac['spend_AED']/new_cac['num_paying_user']

In [30]:
new_cac.to_csv('new_cac.csv')

In [31]:
# get each channel variables
def separate_each_channel(df, channel_list):
    df_list = []
    for i in channel_list:
        n_df = df[df.channel == i].sort_values('spend_AED')
        # calculate marginal_spend
        n_df['marginal_spend']= n_df['spend_AED'].iloc[[0]].append(n_df['spend_AED'].diff().iloc[1:8])
        # calculate marginal_conversions
        n_df['marginal_num_paying_user']= n_df['num_paying_user'].iloc[[0]].append(n_df['num_paying_user'].diff().iloc[1:8])
        # calculate marginal CAC
        n_df['marginal_CAC']= n_df['marginal_spend']/n_df['marginal_num_paying_user']
        df_list.append(n_df)
    return df_list

In [32]:
channel_list = ['bing','display','facebook','search','youtube']
df_list = separate_each_channel(df = new_cac, channel_list = channel_list)


In [33]:
bing = df_list[0]
bing

,channel,spend_AED,month_year,num_paying_user,month_cac,marginal_spend,marginal_num_paying_user,marginal_CAC
25,bing,300,2019-12,13,23.076923,300.0,13.0,23.076923
0,bing,400,2019-07,25,16.000000,100.0,12.0,8.333333
5,bing,900,2019-08,30,30.000000,500.0,5.0,100.000000
10,bing,1000,2019-09,51,19.607843,100.0,21.0,4.761905
15,bing,1100,2019-10,53,20.754717,100.0,2.0,50.000000
20,bing,1300,2019-11,60,21.666667,200.0,7.0,28.571429
30,bing,2100,2020-01,93,22.580645,800.0,33.0,24.242424
35,bing,3700,2020-02,181,20.441989,1600.0,88.0,18.181818


In [34]:
bing.to_csv('bing.csv')

In [35]:
display = df_list[1]
display

,channel,spend_AED,month_year,num_paying_user,month_cac,marginal_spend,marginal_num_paying_user,marginal_CAC
1,display,12,2019-07,7,1.714286,12.0,7.0,1.714286
11,display,13,2019-09,8,1.625000,1.0,1.0,1.000000
16,display,19,2019-10,28,0.678571,6.0,20.0,0.300000
6,display,20,2019-08,33,0.606061,1.0,5.0,0.200000
21,display,29,2019-11,65,0.446154,9.0,32.0,0.281250
26,display,31,2019-12,65,0.476923,2.0,0.0,inf
36,display,94,2020-02,189,0.497354,63.0,124.0,0.508065
31,display,148,2020-01,326,0.453988,54.0,137.0,0.394161


In [36]:
display.to_csv('display.csv')

In [37]:
facebook = df_list[2]
facebook

,channel,spend_AED,month_year,num_paying_user,month_cac,marginal_spend,marginal_num_paying_user,marginal_CAC
2,facebook,9000,2019-07,2058,4.373178,9000.0,2058.0,4.373178
12,facebook,10500,2019-09,3754,2.797017,1500.0,1696.0,0.884434
32,facebook,11000,2020-01,4103,2.680965,500.0,349.0,1.432665
7,facebook,13000,2019-08,4857,2.676549,2000.0,754.0,2.652520
37,facebook,14000,2020-02,4902,2.855977,1000.0,45.0,22.222222
27,facebook,16000,2019-12,6035,2.651201,2000.0,1133.0,1.765225
17,facebook,17000,2019-10,6311,2.693709,1000.0,276.0,3.623188
22,facebook,23000,2019-11,8497,2.706838,6000.0,2186.0,2.744739


In [38]:
facebook.to_csv('facebook.csv')

In [39]:
search = df_list[3]
search

,channel,spend_AED,month_year,num_paying_user,month_cac,marginal_spend,marginal_num_paying_user,marginal_CAC
3,search,13000,2019-07,457,28.446389,13000.0,457.0,28.446389
8,search,18500,2019-08,1179,15.691264,5500.0,722.0,7.617729
13,search,19000,2019-09,1309,14.514897,500.0,130.0,3.846154
18,search,24000,2019-10,1684,14.251781,5000.0,375.0,13.333333
23,search,25000,2019-11,1866,13.397642,1000.0,182.0,5.494505
28,search,38000,2019-12,2701,14.068863,13000.0,835.0,15.568862
33,search,41000,2020-01,2862,14.325646,3000.0,161.0,18.633540
38,search,44000,2020-02,2959,14.869888,3000.0,97.0,30.927835


In [40]:
search.to_csv('search.csv')

In [41]:
youtube = df_list[4]
youtube

,channel,spend_AED,month_year,num_paying_user,month_cac,marginal_spend,marginal_num_paying_user,marginal_CAC
4,youtube,90,2019-07,1,90.000000,90.0,1.0,90.000000
14,youtube,100,2019-09,1,100.000000,10.0,0.0,inf
19,youtube,130,2019-10,7,18.571429,30.0,6.0,5.000000
9,youtube,180,2019-08,11,16.363636,50.0,4.0,12.500000
24,youtube,550,2019-11,36,15.277778,370.0,25.0,14.800000
29,youtube,900,2019-12,56,16.071429,350.0,20.0,17.500000
39,youtube,2420,2020-02,159,15.220126,1520.0,103.0,14.757282
34,youtube,4360,2020-01,312,13.974359,1940.0,153.0,12.679739


In [42]:
youtube.to_csv('youtube.csv')

# Channel marketing optimazition
- Assuming the total budget is 50,000 AED, and we can not spend more than the channel's highest historical month cost, how to spend it to get the most paying users?

# 50,000 AED

- 148 AED on display: get 326 users
- 23,000 AED on facebook, get 8497 users
- 400 AED on bing, get 25 users,
- the rest 26452 on search get 1866 + expect(11452/15.57) = 2601 users

- total users = 11449

- comparision: spend 148 on display, 23,000 on facebook, the rest 26852 on search:
- expect users = 326+8497+26852/14.07 = 10731

conversion lift = (11449-10731)/10731 = 6.7%
CAC = ( 50000/11449 - 50000/10731 ) -1 = - 1.29%